<a href="https://colab.research.google.com/github/xelothi/C-RAF_chemical_analysis/blob/main/auto_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, Descriptors, rdFingerprintGenerator, AllChem, MACCSkeys
from pycaret.classification import *

ModuleNotFoundError: ignored

In [ ]:
df = pd.read_csv("CRAF_cleaned_and_preprocessed_data.csv", index_col=[0])
df

,molecule_chembl_id,std_smiles,MW,LogP,NumHDonors,NumHAcceptors,numRotatingBonds,norm_value,pIC50,bioactivity
0,CHEMBL18754,CNC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccc(C)cc1,345.468,4.35762,3.0,3.0,3.0,4900.00,5.309804,active
1,CHEMBL279560,COC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccc(O)cc1,348.424,4.18180,3.0,5.0,3.0,15000.00,4.823909,active
2,CHEMBL95114,COC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccn(C(C)C)n1,364.471,4.25360,2.0,6.0,4.0,16000.00,4.795880,active
3,CHEMBL279377,COC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccc(C)cc1,346.452,4.78462,2.0,4.0,3.0,1700.00,5.769551,active
4,CHEMBL95692,COc1nccc(-c2c(-c3ccc(F)cc3)ncn2C2CCNCC2)n1,353.401,3.07930,1.0,6.0,4.0,50.00,7.301030,inactive
...,...,...,...,...,...,...,...,...,...,...
1699,CHEMBL5088891,CC1(C)CN(CCNc2nccc(-c3c(-c4cccc(O)c4)nc4sccn34...,485.595,2.56590,3.0,9.0,6.0,4.26,8.370590,inactive
1700,CHEMBL5079413,CN1CCCN(CCNc2nccc(-c3c(-c4cccc(O)c4)nc4sccn34)...,485.595,2.51960,2.0,9.0,6.0,1.96,8.707744,inactive
1701,CHEMBL5076688,O=S1(=O)NCCN1CCNc1nccc(-c2c(-c3cccc(O)c3)nc3sc...,457.541,1.78730,3.0,9.0,6.0,6.32,8.199283,inactive
1702,CHEMBL5079502,O=S1(=O)NCCCN1CCNc1nccc(-c2c(-c3cccc(O)c3)nc3s...,471.568,2.17740,3.0,9.0,6.0,7.21,8.142065,inactive


In [ ]:
df['activity'] = df['bioactivity'].apply(lambda x: 1 if x == 'active' else 0)
df

NameError: ignored

In [ ]:
def fp(molecule, fingerprint_type, n_bits=32784):
    mol = Chem.MolFromSmiles(molecule)
    if fingerprint_type == "RDKFingerprints":
        return np.array(Chem.RDKFingerprint(mol, maxPath=5))
    elif fingerprint_type == "Morgan":
        return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits, useFeatures=True))
    else:
        return np.array(MACCSkeys.GenMACCSKeys(mol))



In [ ]:
df["RDK_fingerprints"] = df["std_smiles"].apply(lambda x: fp(x, "RDKFingerprints"))
df["Morgan"] = df["std_smiles"].apply(lambda x: fp(x, "Morgan"))
df["MACCS"] = df["std_smiles"].apply(lambda x: fp(x, "MACCS"))

In [ ]:
df

,molecule_chembl_id,std_smiles,MW,LogP,NumHDonors,NumHAcceptors,numRotatingBonds,norm_value,pIC50,bioactivity,activity,RDK_fingerprints,Morgan,MACCS
0,CHEMBL18754,CNC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccc(C)cc1,345.468,4.35762,3.0,3.0,3.0,4900.00,5.309804,active,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,CHEMBL279560,COC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccc(O)cc1,348.424,4.18180,3.0,5.0,3.0,15000.00,4.823909,active,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,CHEMBL95114,COC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccn(C(C)C)n1,364.471,4.25360,2.0,6.0,4.0,16000.00,4.795880,active,1,"[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,CHEMBL279377,COC(=O)c1sc(C(C)(C)C)cc1NC(=O)Nc1ccc(C)cc1,346.452,4.78462,2.0,4.0,3.0,1700.00,5.769551,active,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,CHEMBL95692,COc1nccc(-c2c(-c3ccc(F)cc3)ncn2C2CCNCC2)n1,353.401,3.07930,1.0,6.0,4.0,50.00,7.301030,inactive,0,"[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,CHEMBL5088891,CC1(C)CN(CCNc2nccc(-c3c(-c4cccc(O)c4)nc4sccn34...,485.595,2.56590,3.0,9.0,6.0,4.26,8.370590,inactive,0,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1700,CHEMBL5079413,CN1CCCN(CCNc2nccc(-c3c(-c4cccc(O)c4)nc4sccn34)...,485.595,2.51960,2.0,9.0,6.0,1.96,8.707744,inactive,0,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...","[1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1701,CHEMBL5076688,O=S1(=O)NCCN1CCNc1nccc(-c2c(-c3cccc(O)c3)nc3sc...,457.541,1.78730,3.0,9.0,6.0,6.32,8.199283,inactive,0,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1702,CHEMBL5079502,O=S1(=O)NCCCN1CCNc1nccc(-c2c(-c3cccc(O)c3)nc3s...,471.568,2.17740,3.0,9.0,6.0,7.21,8.142065,inactive,0,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Auto ML with RDKFingerprints

In [ ]:
selection = ["RDK_fingerprints", "activity"]
RDKFingeprints_df = df[selection]
RDKFingeprints_df = RDKFingeprints_df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)

NameError: ignored

In [ ]:
RDKFingeprints_df

,RDK_fingerprints,activity
0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",1
1,"(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...",1
2,"(1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...",1
3,"(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...",1
4,"(1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...",0
...,...,...
1699,"(1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...",0
1700,"(1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...",0
1701,"(1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...",0
1702,"(1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...",0


In [ ]:
exp = ClassificationExperiment()

In [ ]:
setup(data=RDKFingeprints_df, target = 'activity', session_id=123)

,Description,Value
0,Session id,123
1,Target,activity
2,Target type,Binary
3,Original data shape,"(1704, 2)"
4,Transformed data shape,"(1704, 2)"
5,Transformed train set shape,"(1192, 2)"
6,Transformed test set shape,"(512, 2)"
7,Categorical features,1
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best_RDKFingerprints = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:06:25
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Naive Bayes


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6611,0.5070,0.0000,0.0000,0.0000,0.0000,0.0000,66.2670
knn,K Neighbors Classifier,0.5977,0.5057,0.2000,0.0683,0.1019,0.0000,0.0000,88.5300


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6611,0.5070,0.0000,0.0000,0.0000,0.0000,0.0000,66.2670
nb,Naive Bayes,0.6518,0.4930,0.0000,0.0000,0.0000,-0.0181,-0.0480,56.6460
knn,K Neighbors Classifier,0.5977,0.5057,0.2000,0.0683,0.1019,0.0000,0.0000,88.5300


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:06:25
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Decision Tree Classifier


KeyboardInterrupt: ignored

In [ ]:
final_model = create_model(best)

NameError: ignored

In [ ]:
plot_model(best, "auc")

NameError: ignored

#AutoML with Morgan fingerprints

*   List item
*   List item



In [ ]:
selection = ["Morgan", "activity"]
Morgan_df = df[selection]
Morgan_df = Morgan_df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)

In [ ]:
setup(data=dfdf, target = 'activity', session_id=123)

In [ ]:
exp = ClassificationExperiment()
best_model = compare_models()

In [ ]:
plot_model(best_model, "auc")